# KIAM8 : WEEK 2 

Task 3: Store Cleaned Data in PostgreSQL

Design and implement a relational database in PostgreSQL to persistently store the cleaned and processed review data. 

This step simulates real-world data engineering workflows, where PostgreSQL is widely used for its robustness, ease of use, and strong community support.

## Sub Tasks

●	PostgreSQL Database Setup

○	Install PostgreSQL on your system.

○	PostgreSQL is generally straightforward to install across different operating systems. If you encounter issues, consult facilitators or refer to the official documentation.

### Define Schema

Banks Table: Stores information about the banks.

Suggested columns: bank_id (PRIMARY KEY), bank_name, app_name


In [1]:
# Import important libraries

from sqlalchemy import create_engine,text
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [2]:
# Define your connection details
DB_HOST = "localhost"
DB_NAME = "bank_reviews"
DB_USER = "postgres"
DB_PASS = "12345678"
DB_PORT = "5432"

# 1. Create the connection URL
DB_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# 2. Create the SQLAlchemy Engine
# The 'echo=False' prevents logging all generated SQL statements to the console
engine = create_engine(DB_URL, echo=False) 

print(f"SQLAlchemy Engine created for {DB_NAME}.")

SQLAlchemy Engine created for bank_reviews.


In [4]:
query_create_banks_table = """
CREATE TABLE banks_table (
    bank_id VARCHAR(50) PRIMARY KEY,
    bank_name VARCHAR(100) NOT NULL,
    app_name VARCHAR(100)
);
"""

try:
    with engine.connect() as connection:
        with connection.begin():
            # Use connection.execute()
            connection.execute(text(query_create_banks_table))
    print("✅ Successfully Created Banks Table")

except Exception as e:
    print(f"❌ Error during table creation: {e}")

✅ Successfully Created Banks Table


In [5]:
banks_data = [('CBE','Commercial Bank of Ethiopia','com.combanketh.mobilebanking'),
			 ('AWASH','Awash Bank', 'com.sc.awashpay'),
			 ('BOA','Abyssinia Bank','com.boa.boaMobileBanking')]

banks_df = pd.DataFrame(
    banks_data, 
    columns=['bank_id', 'bank_name', 'app_name']
)

banks_df.head()

,bank_id,bank_name,app_name
0,CBE,Commercial Bank of Ethiopia,com.combanketh.mobilebanking
1,AWASH,Awash Bank,com.sc.awashpay
2,BOA,Abyssinia Bank,com.boa.boaMobileBanking


In [6]:
# Parameters for the to_sql function:
table_name = "banks_table"  # The name of the table in your Postgres database

# Create a temporary connection and commit explicitly

try:
    with engine.connect() as connection:
        with connection.begin(): # Starts a transaction block
            banks_df.to_sql(
                name=table_name,
                con=connection, # Use the connection object
                if_exists='append',
                index=False,
                method='multi'
            )
            print(f"Data successfully loaded into the '{table_name}' table.")
# Changes are committed upon successful exit of the second 'with' block.

except Exception as e:
    print(f"An error occurred during data loading: {e}")

Data successfully loaded into the 'banks_table' table.


In [7]:
check_query = """ SELECT * FROM banks_table """

try:
    # Use pd.read_sql() to execute the query and convert the result to a DataFrame
    banks_df_from_db = pd.read_sql(check_query, engine)
    
    print(f"Data successfully retrieved. Rows: {len(banks_df_from_db)}")
    # Display the DataFrame
    banks_df_from_db
    
except Exception as e:
    print(f"❌ Error retrieving data: {e}")

Data successfully retrieved. Rows: 3


In [8]:
# Examine the Banks Table
banks_df_from_db.head()

,bank_id,bank_name,app_name
0,CBE,Commercial Bank of Ethiopia,com.combanketh.mobilebanking
1,AWASH,Awash Bank,com.sc.awashpay
2,BOA,Abyssinia Bank,com.boa.boaMobileBanking


In [9]:
# Import the Cleaned & Analyzed Sentiment Data From Task 2

df=pd.read_csv('../data/task2_results.csv')

In [10]:
#Examine the Imported Data

df.head()

,reviewId,review,rating,date,bank,source,sentiment_label,sentiment_score,clean_review,theme_cluster,identified_theme
0,28f229b5-0026-41b9-a1eb-b76e74736f63,Make it user friendly.,2,2025-11-29,CBE,Google Play,positive,0.992062,make it user friendly.,3,Customer Support
1,68d8daea-db47-4e23-a692-755173dea983,maaliif daddafee install gaafata,3,2025-11-28,CBE,Google Play,negative,0.987553,maaliif daddafee install gaafata,3,Customer Support
2,ee0dbb0e-4eb0-47b5-9874-c37877493f99,good app,5,2025-11-28,CBE,Google Play,positive,0.999849,good app,3,Customer Support
3,5112423d-e618-44ba-ba49-62677cb76cd6,This application is very important and advanta...,5,2025-11-27,CBE,Google Play,positive,0.998468,this application is very important and advanta...,3,Customer Support
4,bcb34681-1dd4-4781-b400-4393bb10b1d9,why didn't work this app?,1,2025-11-27,CBE,Google Play,negative,0.999198,why didnt work this app?,3,Customer Support


In [16]:
query_create_reviews_table = """
CREATE TABLE reviews_table (
review_id VARCHAR(50) PRIMARY KEY, 
bank_id VARCHAR(50) REFERENCES banks_table(bank_id),
review_text VARCHAR, 
rating INT,
review_date DATE,
sentiment_label VARCHAR,
sentiment_score FLOAT,
source VARCHAR
);
"""

try:
    with engine.connect() as connection:
        with connection.begin():
            # Use connection.execute()
            connection.execute(text(query_create_reviews_table))
    print("✅ Successfully Created Reviews Table")

except Exception as e:
    print(f"❌ Error during table creation: {e}")

✅ Successfully Created Reviews Table


In [34]:
# Clean & Normalize the Namings on the Bank Colummn

def adjuster(bank):
    if bank == "Abyssinia Bank":
        return "BOA"
    elif bank == "Awash Bank":
        return "AWASH"
    else:
        return "CBE"
    
# Save the Cleaned Data under new DF

df_clean=df.copy()

df_clean['bank']=df['bank'].apply(adjuster) 

# Select Columns on the Cleanded DF to match the Reviews_Table

df_clean=df_clean[['reviewId','bank','review','date','sentiment_label','sentiment_score','source']]
df_clean.rename(columns={'reviewId':'review_id','bank':'bank_id','review':'review_text','date':'review_date'},inplace=True)

In [35]:
# Examine th Clean Dataframe
df_clean.head()

,review_id,bank_id,review_text,review_date,sentiment_label,sentiment_score,source
0,28f229b5-0026-41b9-a1eb-b76e74736f63,CBE,Make it user friendly.,2025-11-29,positive,0.992062,Google Play
1,68d8daea-db47-4e23-a692-755173dea983,CBE,maaliif daddafee install gaafata,2025-11-28,negative,0.987553,Google Play
2,ee0dbb0e-4eb0-47b5-9874-c37877493f99,CBE,good app,2025-11-28,positive,0.999849,Google Play
3,5112423d-e618-44ba-ba49-62677cb76cd6,CBE,This application is very important and advanta...,2025-11-27,positive,0.998468,Google Play
4,bcb34681-1dd4-4781-b400-4393bb10b1d9,CBE,why didn't work this app?,2025-11-27,negative,0.999198,Google Play


In [26]:
df_clean.shape

(1479, 7)

Insert cleaned review data using Python

In [36]:
# Parameters for the to_sql function:
New_table_name = "reviews_table"  # The name of the table in your Postgres database

# Create a temporary connection and commit explicitly

try:
    with engine.connect() as connection:
        with connection.begin(): # Starts a transaction block
            df_clean.to_sql(
                name=New_table_name,
                con=connection, # Use the connection object
                if_exists='append',
                index=False,
                method='multi'
            )
            print(f"Data successfully loaded into the '{table_name}' table.")
# Changes are committed upon successful exit of the second 'with' block.

except Exception as e:
    print(f"An error occurred during data loading: {e}")

Data successfully loaded into the 'banks_table' table.


Write SQL queries to verify data integrity (e.g., count reviews per bank, average rating)

In [37]:
sql_query = """
    SELECT 
        *
    FROM 
        REVIEWS_TABLE
    LIMIT 10;
"""

try:
    # Use pandas read_sql() with the SQLAlchemy engine
    # This automatically opens, executes, and closes the connection/transaction
    results_df = pd.read_sql(sql_query, engine)
    
    print(f"Query executed successfully. Rows returned: {len(results_df)}")
    
    # Display the results in the notebook
    results_df
    
except Exception as e:
    print(f"An error occurred: {e}")

Query executed successfully. Rows returned: 10


In [38]:
results_df.head()

,review_id,bank_id,review_text,rating,review_date,sentiment_label,sentiment_score,source
0,28f229b5-0026-41b9-a1eb-b76e74736f63,CBE,Make it user friendly.,None,2025-11-29,positive,0.992062,Google Play
1,68d8daea-db47-4e23-a692-755173dea983,CBE,maaliif daddafee install gaafata,None,2025-11-28,negative,0.987553,Google Play
2,ee0dbb0e-4eb0-47b5-9874-c37877493f99,CBE,good app,None,2025-11-28,positive,0.999849,Google Play
3,5112423d-e618-44ba-ba49-62677cb76cd6,CBE,This application is very important and advanta...,None,2025-11-27,positive,0.998468,Google Play
4,bcb34681-1dd4-4781-b400-4393bb10b1d9,CBE,why didn't work this app?,None,2025-11-27,negative,0.999198,Google Play


# Key take Aways

Created a dabatabse in Postgress Named Bank_Reviews

Established a connection with the local database using Sql Alchemy

Created Tables Named Banks & Reviews

Pouplated the Banks Table with Banks Data & Populated the Reviews Table with the Cleaned Data From Task 2